In [ ]:
!pip install unsloth==2025.2.8 bitsandbytes torch numpy tf-keras transformers peft accelerate datasets matplotlib tensorflow scikit-learn
!pip install triton
!pip uninstall unsloth unsloth_zoo -y
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --upgrade --no-cache-dir "git+https://github.com/unslothai/unsloth-zoo.git"
!pip install --upgrade transformers

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import pandas as pd
import matplotlib.pyplot as plt
import os
from transformers import TextStreamer, TrainingArguments
from datasets import load_dataset
from trl import SFTTrainer
from google.colab import userdata

In [ ]:
## Prompt Configuration
prompt = """Below is an instruction describing a test, paired with an input that provides the test ID and the field being tested.
The response should contain a Python function that performs the test, using the exact test description as a docstring.
Do not modify the provided field names under any circumstances.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
## Load Model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

In [ ]:
# Reformat .xlsx training dataset
data = pd.read_excel("train_data.xlsx")
dataset = data.to_dict(orient="records")
print(dataset)
import json

with open("train.jsonl","w") as f:
  for line in dataset:
    f.write(json.dumps(line) + "\n")

In [ ]:
## Load training data (with validation dataset)
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = load_dataset("cvegas/Llama_training_data_V6", token=userdata.get('HFTOKEN'), split="train")
train_val_dataset = dataset.train_test_split(test_size=0.02, shuffle=True)
train_dataset = train_val_dataset['train']
validation_dataset = train_val_dataset['test']

train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
validation_dataset = validation_dataset.map(formatting_prompts_func, batched=True)

In [ ]:
## Training
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    use_gradient_checkpointing = "unsloth",
    use_rslora = True,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = validation_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        per_device_eval_batch_size = 1,
        gradient_accumulation_steps = 4,
        evaluation_strategy = "steps",
        warmup_steps = 1,
        max_steps = 200,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        eval_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 0,
        output_dir = "outputs",
        report_to="none"
    ),
)
trainer.train()

In [ ]:
## Traning Loss Evolution Plot
train_losses=[]
eval_losses=[]
train_steps=[]
eval_steps=[]

for entry in trainer.state.log_history:
  if 'loss' in entry:
    train_losses.append(entry['loss'])
    train_steps.append(entry['step'])
  if 'eval_loss' in entry:
    eval_losses.append(entry['eval_loss'])
    eval_steps.append(entry['step'])

plt.plot(train_steps, train_losses, label='Train Loss')
plt.plot(eval_steps, eval_losses, label='Eval Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
## Post-Training Performance
instruction = ""
input = ""

FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt.format(instruction, input, "")], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2000)

In [ ]:
# Save model and tokenizer locally
model.save_pretrained("Llama_3.1_finetune_V6")
tokenizer.save_pretrained("Llama_3.1_finetune_V6")

# Push model and tokenizer to Huggingface Hub
huggingface_model_name = "cvegas/Llama_3.1_finetune_V6"
model.push_to_hub(huggingface_model_name, token=userdata.get('HFTOKEN'))
tokenizer.push_to_hub(huggingface_model_name, token=userdata.get('HFTOKEN'))

# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method="merged_16bit",)
if False: model.push_to_hub_merged(huggingface_model_name, tokenizer, save_method="merged_16bit", token=userdata.get('HFTOKEN'))

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method="merged_4bit",)
if False: model.push_to_hub_merged(huggingface_model_name, tokenizer, save_method="merged_4bit", token=userdata.get('HFTOKEN'))

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method="lora",)
if False: model.push_to_hub_merged(huggingface_model_name, tokenizer, save_method="lora", token=userdata.get('HFTOKEN'))

# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer)
if False: model.push_to_hub_gguf(huggingface_model_name, tokenizer, token=userdata.get('HFTOKEN'))

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method="f16")
if False: model.push_to_hub_gguf(huggingface_model_name, tokenizer, quantization_method="f16", token=userdata.get('HFTOKEN'))

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method="q4_k_m")
if False: model.push_to_hub_gguf(huggingface_model_name, tokenizer, quantization_method="q4_k_m", token=userdata.get('HFTOKEN'))

In [ ]:
## Huggingface Model Inference
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "cvegas/Llama_3.1_finetune_V6",
    token = userdata.get('HFTOKEN'),
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

In [ ]:
FastLanguageModel.for_inference(model)
text_streamer = TextStreamer(tokenizer)
data = pd.read_excel("new_tests.xlsx")

for index, row in data.iterrows():
    instruction = data.at[index,'instruction']
    input = data.at[index,'input']
    inputs = tokenizer([prompt.format(instruction, input, "",)], return_tensors = "pt").to("cuda")
    _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2000)